In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import xgboost as xgb

In [3]:
equity_df = pd.read_csv('equity_value_data.csv')

In [4]:
features_df = pd.read_csv('features_data.csv')

In [5]:
equity_df.sample(5)

,timestamp,close_equity,user_id
1085864,2016-11-25T00:00:00Z,100.000,9f037766af4641eec3cd506b7feb0cf2
376066,2016-11-02T00:00:00Z,27.160,c96724b6ba588a8ebaa00e44f48d599f
1009477,2017-06-30T00:00:00Z,89.520,8b3338eaefeb87be1adf80f019b887da
1009528,2017-02-27T00:00:00Z,106.780,b7d15f451e6b7fb2fcaa449f74c6dbdb
599252,2017-06-16T00:00:00Z,632.482,58d1a7f78a01a1b7b2a2f3ce17613d83


In [6]:
features_df.sample(5)

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
160,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,9.107717,stock,50.0,med_time_horizon,36a7ba903fcc3ed356276d5342bc933b
1032,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,33.820500,stock,1500.0,long_time_horizon,fede0937ab0dbff7d239afe022e1266f
4040,med_risk_tolerance,no_investment_exp,very_important_liq_need,iOS,5.283867,stock,10.0,short_time_horizon,c9ffc42d3e26472b8cc39e9cbb02fee7
2121,low_risk_tolerance,no_investment_exp,very_important_liq_need,Android,88.348050,stock,20.0,med_time_horizon,e4dfa0cc334f9a710801f0728040e894
5305,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,51.549800,stock,2.0,med_time_horizon,08e2b47ff3aa643120e99ace5f1594b9


In [7]:
equity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   timestamp     1119158 non-null  object 
 1   close_equity  1119158 non-null  float64
 2   user_id       1119158 non-null  object 
dtypes: float64(1), object(2)
memory usage: 25.6+ MB


In [8]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   risk_tolerance                5584 non-null   object 
 1   investment_experience         5584 non-null   object 
 2   liquidity_needs               5584 non-null   object 
 3   platform                      5584 non-null   object 
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object 
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object 
 8   user_id                       5584 non-null   object 
dtypes: float64(2), object(7)
memory usage: 392.8+ KB


# **What percentage of users have churned in the data provided?**

In [9]:
equity_df['date'] = pd.to_datetime(equity_df['timestamp']).dt.tz_localize(None)

In [10]:
equity_df.drop('timestamp', axis = 1, inplace = True)

In [11]:
equity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   close_equity  1119158 non-null  float64       
 1   user_id       1119158 non-null  object        
 2   date          1119158 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.6+ MB


In [12]:
equity_df.sort_values(['user_id', 'date']).head(20)

,close_equity,user_id,date
505994,1211.6055,0012db34aa7b083f5714e7831195e54d,2016-08-18
505995,1173.5640,0012db34aa7b083f5714e7831195e54d,2016-08-19
505996,1253.0597,0012db34aa7b083f5714e7831195e54d,2016-08-22
505997,1252.9050,0012db34aa7b083f5714e7831195e54d,2016-08-23
505998,1262.1360,0012db34aa7b083f5714e7831195e54d,2016-08-24
505999,1226.4250,0012db34aa7b083f5714e7831195e54d,2016-08-25
506000,1227.9610,0012db34aa7b083f5714e7831195e54d,2016-08-26
506001,1250.9700,0012db34aa7b083f5714e7831195e54d,2016-08-29
506002,1248.1000,0012db34aa7b083f5714e7831195e54d,2016-08-30
506003,1241.5560,0012db34aa7b083f5714e7831195e54d,2016-08-31


In [13]:
equity_df.groupby('user_id')['date'].agg(['min','max']).sort_values('min')

,min,max
user_id,,
f432a3b953f835df27db9097306b4b6b,2016-08-16,2017-08-15
f3664d2f94b202d9c9216528c0c5ceb9,2016-08-16,2017-08-15
5a0ebe0c4e160ce7b293479ccd2cb928,2016-08-16,2017-08-15
e166923069abfc7616c130e8e877a8c2,2016-08-16,2017-08-15
6dd369e1c9685a5147451b3c8e58087d,2016-08-16,2017-08-15
...,...,...
11e831fb3f1da3e4a4f3353fa43283a6,2017-07-24,2017-08-10
46b308ea2f5f96bf6f53c62cd65bcc8d,2017-08-03,2017-08-17
404f3e9bf87e98f0c54795e71bef2f69,2017-08-04,2017-08-17


In [14]:
end_date = pd.to_datetime('2017-08-18')

In [15]:
results = []
for user_id, group in equity_df.groupby('user_id'):
    date_range = pd.date_range(group['date'].min(), end_date)
    group = group.set_index('date').reindex(date_range)
    group['user_id'] = user_id
    group['close_equity'] = group['close_equity'].fillna(0)
    results.append(group)

In [16]:
filled_equity_df = pd.concat(results).reset_index().rename(columns = {'index':'date'})
filled_equity_df

,date,close_equity,user_id
0,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d
1,2016-08-19,1173.5640,0012db34aa7b083f5714e7831195e54d
2,2016-08-20,0.0000,0012db34aa7b083f5714e7831195e54d
3,2016-08-21,0.0000,0012db34aa7b083f5714e7831195e54d
4,2016-08-22,1253.0597,0012db34aa7b083f5714e7831195e54d
...,...,...,...
1754501,2017-08-14,2156.2400,ffc1e622f3a0b2666f09a6dcb7f27918
1754502,2017-08-15,2134.7100,ffc1e622f3a0b2666f09a6dcb7f27918
1754503,2017-08-16,2152.1200,ffc1e622f3a0b2666f09a6dcb7f27918
1754504,2017-08-17,2042.2800,ffc1e622f3a0b2666f09a6dcb7f27918


In [17]:
filled_equity_df.isnull().sum()

date            0
close_equity    0
user_id         0
dtype: int64

In [18]:
filled_equity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754506 entries, 0 to 1754505
Data columns (total 3 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   close_equity  float64       
 2   user_id       object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 40.2+ MB


In [19]:
user_with_10_filter = filled_equity_df.groupby('user_id')['close_equity'].max()
user_with_10 = user_with_10_filter[user_with_10_filter >=10].index
user_with_10

Index(['0012db34aa7b083f5714e7831195e54d', '001d6c77dbdb3213cead7673f250bfdc',
       '002e4653171ddc61c3cd30603cd7bd3e', '00384fa9be6fdca1b786bae70d78f88f',
       '0042aac295a0d4df88f4b83012778bd4', '00440034cc4152bfb01b30f5c381c4e3',
       '004aab1640f3a04b87b1f404fb4c018d', '004ea9d7662aa8dc840bbff212cfa4b8',
       '005d630a68b4ab3a2f4cd49d9a87c50d', '00a9602e72eb34cb17aaad6ce62730b6',
       ...
       'ff73b9d5b654cad407b5b233dc5de171', 'ff7610fdd7ac5cbfa0b17aca53af5db4',
       'ff8e6d549f1244d4ef7bbf41346170e5', 'ff934389faba73002fb94daf1f7dcfc9',
       'ff98cbc9633a9255a711d02ebc1e8294', 'ff9ee08791e20724a86995ab2bc72578',
       'ffa12d2f97e310910291f9b26fb2318d', 'ffae713096867a32e74f633060667153',
       'ffbda9a14e07718e2b21fb03896d21c5', 'ffc1e622f3a0b2666f09a6dcb7f27918'],
      dtype='object', name='user_id', length=5584)

In [20]:
user_with_10_df = filled_equity_df[filled_equity_df['user_id'].isin(user_with_10)]

In [21]:
churned_users = []
for user_id, group in user_with_10_df.groupby('user_id'):
    group = group.sort_values('date')
    streak = 0
    prev_date = None
    for date, equity in zip(group['date'], group['close_equity']):
        if equity < 10:
            if prev_date is not None and (date - prev_date) == pd.Timedelta(days=1):
                streak = streak + 1
            else:
                streak = 1
            if streak >=28:
                churned_users.append(user_id)
                break
        else:
            streak = 0
        prev_date = date

In [22]:
churned_users_count = len(churned_users)
churned_users_count

1005

In [23]:
percent_churned = churned_users_count/equity_df['user_id'].nunique()
percent_churned

0.17997851002865328

# **Build a classifier that, given a user with their features, assigns a churn probability for every user and predicts which users will churn.**

## **Train_Test_Split and ColumnTransform**

In [24]:
features_df['churned'] = features_df['user_id'].isin(churned_users).astype(int)

In [25]:
features_df['churned'].value_counts()

churned
0    4579
1    1005
Name: count, dtype: int64

In [26]:
X = features_df.drop(['user_id', 'churned'], axis = 1)
y = features_df['churned']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [28]:
categorical_features = ['risk_tolerance', 'investment_experience', 'liquidity_needs', 
                        'platform', 'instrument_type_first_traded', 'time_horizon']

In [29]:
numerical_features = ['time_spent', 'first_deposit_amount']

In [30]:
preprocessor = ColumnTransformer(transformers = [('cat', OneHotEncoder(handle_unknown = 'ignore'), categorical_features),
                                                 ('num', 'passthrough', numerical_features)])

## **RandomForestClassifier**

In [31]:
model_1 = Pipeline(steps = [('preprocessor', preprocessor),
                            ('classifier', RandomForestClassifier(n_estimators = 300,
                                                                  max_depth = 8,
                                                                  random_state = 42,
                                                                  class_weight = 'balanced'))])

In [32]:
model_1.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [33]:
y_prob_rf = model_1.predict_proba(X_test)[:,1]
y_pred_rf = model_1.predict(X_test)

In [34]:
print("RandomForest ROC-AUC:", roc_auc_score(y_test, y_prob_rf))

RandomForest ROC-AUC: 0.6410822524929936


In [35]:
print("RandomForest Report", classification_report(y_test, y_pred_rf))

RandomForest Report               precision    recall  f1-score   support

           0       0.86      0.72      0.79       916
           1       0.27      0.46      0.34       201

    accuracy                           0.68      1117
   macro avg       0.56      0.59      0.56      1117
weighted avg       0.75      0.68      0.71      1117



#### **RandomForestClassifier GridSearch**

In [36]:
rf_param_grid = {'classifier__n_estimators':[50, 100, 200, 300, 500],
              'classifier__max_depth':[None, 1, 2, 3, 5, 8, 12],
              'classifier__min_samples_split':[0, 2, 5, 8, 10, 12, 15, 18]}

In [37]:
rf_grid_search = GridSearchCV(estimator = model_1,
                           param_grid = rf_param_grid,
                           cv = 5, 
                           scoring = 'roc_auc',
                           n_jobs = -1,
                           verbose = 2)

In [38]:
rf_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 280 candidates, totalling 1400 fits


C:\Users\sasan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
175 fits failed out of a total of 1400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
175 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sasan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sasan\anaconda3\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\sasan\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 663, in fit
    self._final_estimator

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'classifier__max_depth': [None, 1, ...], 'classifier__min_samples_split': [0, 2, ...], 'classifier__n_estimators': [50, 100, ...]}"
,scoring,'roc_auc'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('cat', ...), ('num', ...)]"


In [39]:
print("RandomForest Best ROC-AUC:", rf_grid_search.best_score_)

RandomForest Best ROC-AUC: 0.6736770487659065


In [40]:
print("RandomForest Best hyperparameters:", rf_grid_search.best_params_)

RandomForest Best hyperparameters: {'classifier__max_depth': 5, 'classifier__min_samples_split': 18, 'classifier__n_estimators': 500}


In [41]:
rf_best_model = rf_grid_search.best_estimator_

In [42]:
y_prob_rf_grid = rf_best_model.predict_proba(X)[:,1]

In [43]:
features_df['churn_probability'] = y_prob_rf_grid

In [44]:
features_df['predicted_churn'] = (y_prob_rf_grid >= 0.5).astype(int)

In [45]:
features_df.sample(20)

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churned,churn_probability,predicted_churn
197,high_risk_tolerance,no_investment_exp,very_important_liq_need,Android,36.546483,stock,500.0,long_time_horizon,8d0ce4d7290a8033257e458ee7dce763,0,0.423897,0
432,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,273.753083,stock,100.0,long_time_horizon,2e9caf1ef7df1fda7004bca0722de7ac,0,0.488021,0
3531,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,0.000000,stock,500.0,long_time_horizon,357c7a88050480b1040acf33dc299282,0,0.289316,0
5457,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,9.542617,stock,100.0,long_time_horizon,acc1b39142757f4d7920267df7d52870,0,0.456483,0
719,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,40.445467,stock,50.0,short_time_horizon,e5e37fd4d74d4ff136e1f44f6a4c8c74,1,0.521447,1
661,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,72.187650,stock,20.0,long_time_horizon,a53faa8b06f90f096c45a6b21f2d79af,0,0.516979,1
4480,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,113.155533,stock,100.0,long_time_horizon,17824f1e38bb56dc4df8dc10184904fe,0,0.482421,0
3029,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,18.331667,stock,100.0,long_time_horizon,d3af1d9d4e542c4d5f1c11e163529eb4,0,0.454185,0
3433,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,0.000000,stock,500.0,long_time_horizon,e18e73f680c997be9f9ebc0db9fa23b3,0,0.289316,0
3306,med_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.363500,stock,50.0,short_time_horizon,902055b6dc962d3d248f49bc1822838b,1,0.559915,1


# **List the most important features that correlate to user churn**

In [46]:
cat_features = rf_grid_search.best_estimator_.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
all_features = list(cat_features) + numerical_features

In [47]:
importances = rf_grid_search.best_estimator_.named_steps['classifier'].feature_importances_

In [48]:
importances_df = pd.DataFrame({'feature':all_features, 'importance':importances})
importances_df = importances_df.sort_values(by = 'importance', ascending = False)

In [49]:
importances_df

,feature,importance
28,first_deposit_amount,0.276454
27,time_spent,0.262323
0,risk_tolerance_high_risk_tolerance,0.110905
2,risk_tolerance_med_risk_tolerance,0.056230
1,risk_tolerance_low_risk_tolerance,0.036015
8,liquidity_needs_somewhat_important_liq_need,0.033888
9,liquidity_needs_very_important_liq_need,0.032104
26,time_horizon_short_time_horizon,0.020137
10,platform_Android,0.016535
6,investment_experience_no_investment_exp,0.015466


import pandas as pd

# Load data
equity_df = pd.read_csv('equity_value_data.csv')
features_df = pd.read_csv('features_data.csv')

# Convert timestamp to datetime (naive, no timezone)
equity_df['date'] = pd.to_datetime(equity_df['timestamp']).dt.tz_localize(None)

# End of observation period
END_DATE = pd.to_datetime("2017-08-18")

# Only consider users who ever had equity >= 10
eligible_users = (
    equity_df.groupby('user_id')['close_equity']
    .max()
    .loc[lambda x: x >= 10]
    .index
)

equity_df = equity_df[equity_df['user_id'].isin(eligible_users)]

churned_users = []

# Process each user individually
for user_id, group in equity_df.groupby('user_id'):
    # Sort dates when equity >= 10
    dates = group['date'].sort_values().unique()
    
    churned = False
    
    # Check gaps between consecutive ≥$10 dates
    for prev_date, next_date in zip(dates[:-1], dates[1:]):
        if (next_date - prev_date).days >= 28:
            churned = True
            break
    
    # Check gap after last ≥$10 date until END_DATE
    if not churned and (END_DATE - dates[-1]).days >= 28:
        churned = True
    
    if churned:
        churned_users.append(user_id)

churned_users_count = len(churned_users)
print(f"Number of churned users: {churned_users_count}")
